In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, f1_score, roc_auc_score, cohen_kappa_score, confusion_matrix 
from spot import *
import bisect

In [ ]:
class DataProcess:
    INVALID_ITEM = -1
    def __init__(self, data_file, data_size, config):
        self.file = data_file
        self.data_size = data_size
        self.fake_eps = config['fake_eps']
        self.min_delay = config['min_delay']
        self.max_delay = config['max_delay']
        self.max_bw = np.log(config['max_bw']*1024*1024)
        self.cong_rate = config['cong_rate']
        self.stat_size = config['stat_size']
        self.avg_weight = config['avg_weight']
        self.link_delay_map = {}
        self.path_delay_map = {}
        self.link_loads_map = {}
        self.valid_delay_map = {}
        self.valid_loads_map = {}
        self.delay_limit_map = {}
        self.quality_map = {}
        self.dataset = []
        self.target_dpids = self._get_target_dpids()
    
    def preprocess(self):
        with open(self.file, 'r+') as fp:
            for _ in range(self.data_size):
                line = fp.readline()
                if not line:
                    break 
                self._get_link_delay(line)
                self._get_data_item(line)
                self._get_link_quality(line)
    
    def _get_target_dpids(self):
        target_dpids = set()
        for ep in self.fake_eps:
            dpid = ep[0]
            target_dpids.add(dpid)
        return target_dpids
    
    def _get_link_delay(self, line):
        res_list = line.split()
        link = ''.join(res_list[:3])
        loads = float(res_list[-1])
        path_delay = float(res_list[-2])
        cs_delay = float(res_list[-3])
        sc_delay = float(res_list[-4])
        link_delay = path_delay - (cs_delay + sc_delay)/2
        if link_delay < self.min_delay:
            link_delay = self.min_delay
        link_delays = self.link_delay_map.setdefault(link, [])
        link_delays.append(link_delay)
        link_loads = self.link_loads_map.setdefault(link, [])
        link_loads.append(loads)
        path_delays = self.path_delay_map.setdefault(link, [])
        path_delays.append(path_delay)
    
    def _get_valid_delay(self, link, delay):
        valid_list = self.valid_delay_map.setdefault(link, [self.max_delay])
        if delay == DataProcess.INVALID_ITEM:
            if len(valid_list) < self.stat_size:
                item_list = self.link_delay_map[link]
                s_loc = -min(len(item_list), self.stat_size)
                valid_delay = np.median(item_list[s_loc:])
            else:
                valid_delay = np.median(valid_list)
        else:
            if len(valid_list) >= self.stat_size:
                valid_list.pop(0)
            valid_list.append(delay)
            if len(valid_list) < len(self.avg_weight):
                valid_delay = sum(valid_list)/len(valid_list)
            else:
                s_loc = -len(self.avg_weight)
                valid_delay = sum([self.avg_weight[i]*d \
                    for i, d in enumerate(valid_list[s_loc:])])
        return valid_delay
        
    def _get_valid_loads(self, link, loads):
        valid_list = self.valid_loads_map.setdefault(link, [0])
        if loads == DataProcess.INVALID_ITEM:
            valid_loads = 0
        else:
            if len(valid_list) >= self.stat_size:
                valid_list.pop(0)
            valid_list.append(loads)
            if len(valid_list) < len(self.avg_weight):
                valid_loads = sum(valid_list)/len(valid_list)
            else:
                s_loc = -len(self.avg_weight)
                valid_loads = sum([self.avg_weight[i]*d \
                    for i, d in enumerate(valid_list[s_loc:])])
        return valid_loads
        
    def _get_link_quality(self, line):
        res_list = line.split()
        link = ''.join(res_list[:3])
        delay = float(res_list[-2])
        loads = float(res_list[-1])
        
        # The key idea of this tricks locates at that, the delay beyond a so called 'max_delay' should
        # make no sense to indicate the link quality, as well as the reliability of the links.
        # Statistic of the median delay of the links of the target dpid, so as to obtain the valid delay
        # which is used to indicated the reliability of the links.
        
        if delay > self.max_delay:
            delay = self._get_valid_delay(link, 
                                         DataProcess.INVALID_ITEM)
            loads = self._get_valid_loads(link,
                                         DataProcess.INVALID_ITEM)
        else:
            delay = self._get_valid_delay(link, delay)
            loads = self._get_valid_loads(link, loads)
            
        tq = 1 - np.tanh(delay/self.max_delay)
        lq = 1 / (1 + np.exp(-(loads-self.max_bw*self.cong_rate)))
        
        # The degree of indicating link quality mainly depends on the delay quality, e.g., 
        # Link A: high loads, low delay
        # Link B: low loads, low delay
        # Link C: high loads, high delay
        # Link D: low loads, high delay 
        # we hat A > B > C > D
        # Eq. Q = (Qt + np.exp(Qt-max_Qt) * Ql) / 2, where max_Qt = 1-np.tanh(0) = 1
        quality = tq * (1 + np.exp(tq - 1) * lq) / 2 
        quality_list = self.quality_map.setdefault(link, [])
        quality_list.append(quality)
    
    def _get_data_item(self, line):
        res_list = line.split()
        link = ''.join(res_list[:3])
        dpid = link.split(':')[0]
        if not dpid in self.target_dpids:
            return 
        path_delay = float(res_list[-2])
        cs_delay = float(res_list[-3])
        sc_delay = float(res_list[-4])
        delay = path_delay - (cs_delay + sc_delay)/2
        if delay < self.min_delay:
            delay = self.min_delay
        if any([x in link for x in self.fake_eps]):
            is_forged_link = True 
        else:
            is_forged_link = False 
        data_item = [cs_delay, sc_delay, delay, is_forged_link]
        self.dataset.append(data_item)
    
    def query_quality_map(self):
        return self.quality_map 
    
    def query_delay_map(self):
        return self.link_delay_map
    
    def query_delay_map2(self):
        return self.path_delay_map
    
    def query_dataset(self):
        return np.array(self.dataset)

In [ ]:
class TopoGuardPlus:
    def __init__(self, delay_map, start_range, test_range, fake_eps):
        self.delay_map = delay_map
        self.start_range = start_range 
        self.test_range = test_range
        self.target_dpids = self._get_target_dpids(fake_eps)
        self.fake_links = self._get_fake_links(fake_eps)
        self.link_delay_queue = self._prepare()
    
    def _get_target_dpids(self, fake_eps):
        target_dpids = set()
        for ep in fake_eps:
            dpid = ep[0]
            target_dpids.add(dpid)
        return target_dpids
    
    def _get_fake_links(self, fake_eps):
        fake_links = []
        for link in self.delay_map.keys():
            for ep in fake_eps:
                if ep in link:
                    fake_links.append(link)
                    break 
        return fake_links 
    
    def _prepare(self):
        link_delay_queue = []
        for i in self.start_range:
            for link, delay_list in self.delay_map.items():
                dpid = link.split(':')[0]
                if dpid in self.target_dpids and \
                    link not in self.fake_links:
                    link_delay_queue.append(delay_list[i])
        return link_delay_queue
    
    def fit(self):
        y_pred = []
        y_true = []
        for i in self.test_range:
            Q1 = np.quantile(self.link_delay_queue, 0.25)
            Q3 = np.quantile(self.link_delay_queue, 0.75)
            thr = Q3 + 3 * (Q3 - Q1)
            for link, delay_list in self.delay_map.items():
                dpid = link.split(':')[0]
                if dpid not in self.target_dpids:
                    continue
                curr_delay = delay_list[i]
                self.link_delay_queue.append(curr_delay)
                pred_flag = 1 if curr_delay > thr else False 
                true_flag = 1 if link in self.fake_links else False
                y_pred.append(pred_flag) 
                y_true.append(true_flag)
        return y_pred, y_true             


In [ ]:
class TrustTopo:
    def __init__(self, delay_map, start_range, test_range, fake_eps):
        self.delay_map = delay_map
        self.start_range = start_range 
        self.test_range = test_range
        self.target_dpids = self._get_target_dpids(fake_eps)
        self.fake_links = self._get_fake_links(fake_eps)
        self.link_delay_queue = self._prepare()
        self.C = 0.5

    def _get_target_dpids(self, fake_eps):
        target_dpids = set()
        for ep in fake_eps:
            dpid = ep[0]
            target_dpids.add(dpid)
        return target_dpids
    
    def _get_fake_links(self, fake_eps):
        fake_links = []
        for link in self.delay_map.keys():
            for ep in fake_eps:
                if ep in link:
                    fake_links.append(link)
                    break 
        return fake_links 
    
    def _prepare(self):
        link_delay_queue = []
        for i in self.start_range:
            for link, delay_list in self.delay_map.items():
                dpid = link.split(':')[0]
                if dpid in self.target_dpids and \
                    link not in self.fake_links:
                    link_delay_queue.append(delay_list[i])
        # sort
        link_delay_queue.sort()
        # strip the ends 
        link_delay_queue.pop(0)
        link_delay_queue.pop(-1)
        return link_delay_queue

    def train(self, delay):
        tmp_queue = np.array(self.link_delay_queue)
        med = np.median(tmp_queue)
        prop = np.sum(tmp_queue > med) / len(tmp_queue)
        delta = max(delay-med, 0)
        thr = med + prop * delta + self.C
        return thr
    
    def fit(self):
        y_pred = []
        y_true = []
        thr = self.train(0)
        for i in self.test_range:
            for link, delay_list in self.delay_map.items():
                dpid = link.split(':')[0]
                if dpid not in self.target_dpids:
                    continue
                curr_delay = delay_list[i]
                pred_flag = True if curr_delay > thr else False 
                true_flag = True if link in self.fake_links else False
                y_pred.append(pred_flag) 
                y_true.append(true_flag)
                # update 
                if not pred_flag:
                    self.link_delay_queue.append(curr_delay)
                    thr = self.train(curr_delay)
                #print('thr: {}, current delay: {}'.format(thr, curr_delay))
        return y_pred, y_true

In [ ]:
# 不同于其它方法，LinkGuard本质上是静态的，所以它的阈值是不更新的。
class LinkGuard:
    def __init__(self, delay_map, train_range, test_range, fake_eps):
        self.delay_map = delay_map
        self.train_range = train_range
        self.test_range = test_range
        self.target_dpids = self._get_target_dpids(fake_eps)
        self.fake_links = self._get_fake_links(fake_eps)
    
    def _get_target_dpids(self, fake_eps):
        target_dpids = set()
        for ep in fake_eps:
            dpid = ep[0]
            target_dpids.add(dpid)
        return target_dpids
    
    def _get_fake_links(self, fake_eps):
        fake_links = []
        for link in self.delay_map.keys():
            for ep in fake_eps:
                if ep in link:
                    fake_links.append(link)
                    break 
        return fake_links 
    
    def train(self):
        link_delay_queue = []
        for i in self.train_range:
            for link, delay_list in self.delay_map.items():
                dpid = link.split(':')[0]
                if dpid in self.target_dpids and \
                    link not in self.fake_links:
                    link_delay_queue.append(delay_list[i])
        Q1 = np.quantile(link_delay_queue, 0.25)
        Q3 = np.quantile(link_delay_queue, 0.75)
        thr = Q3 + 3 * (Q3 - Q1)
        return thr
    
    def fit(self):
        y_pred = []
        y_true = []
        thr = self.train()
        for i in self.test_range:
            for link, delay_list in self.delay_map.items():
                dpid = link.split(':')[0]
                if dpid not in self.target_dpids:
                    continue 
                curr_delay = delay_list[i]
                pred_flag = True if curr_delay > thr else False 
                true_flag = True if link in self.fake_links else False 
                y_pred.append(pred_flag)
                y_true.append(true_flag)
        return y_pred, y_true 
    # def fit(self):
    #     y_pred = []
    #     y_true = []
    #     thr = self.train()
    #     # we don't set it to 10 as the original paper, because the performance is much worse than TTL=1
    #     TTL = 1 
    #     measure_result = {}
    #     pred_result = {}
    #     for i in self.test_range:
    #         for link, delay_list in self.delay_map.items():
    #             dpid = link.split(':')[0]
    #             if dpid not in self.target_dpids:
    #                 continue
    #             measure_result.setdefault(link, [])
    #             pred_result.setdefault(link, [])
    #             curr_delay = delay_list[i]
    #             measure_result[link].append(curr_delay)
    #             if len(measure_result[link]) % TTL == 0:
    #                 measure_dealy = np.median(measure_result[link])
    #                 measure_result[link].clear()
    #                 pred_flag = True if measure_dealy > thr else False
    #                 pred_result[link].extend([pred_flag] * TTL) 
    #             true_flag = True if link in self.fake_links else False
    #             y_true.append(true_flag)
    #     # In python 3.9, the keys are fix-ordered in dict.
    #     batch_size = len(self.test_range)
    #     for i in range(batch_size):
    #         for v in pred_result.values():
    #             y_pred.append(v[i])
    #     assert(len(y_pred) == len(y_true)) 
    #     return y_pred, y_true      

In [ ]:
def show_model_metrics(model_name, y_true, y_pred):   
    cnf_matrix = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = cnf_matrix.ravel()
    
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    
    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)
    
    Pr = precision_score(y_true, y_pred)
    F1 = f1_score(y_true, y_pred)
    
    # Error, the 2nd param of roc_auc_score is not y_pred
    # but y_score, which is the probability of y_pred 
    AUC = roc_auc_score(y_true, y_pred)
    print(f'''
          name : {model_name},
          TPR: {TPR},
          FPR: {FPR},
          Precision: {Pr},
          f1_score: {F1},
          auc_score: {AUC}
          ''')
    return TPR, FPR, Pr, F1, AUC 

In [ ]:
def query_model_predict(model, model_name, dataset):
    dataset = np.array(dataset)
    X, y = dataset[:,:-1], dataset[:,-1]
    multi_step_model = ['lr', 'rf', 'svc', 'mlp']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.334, random_state=0)
    clf = model.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    if model_name in multi_step_model:
        y_pred[y_pred > 0.5] = 1
        y_pred[y_pred <= 0.5] = 0
    return y_pred, y_test  

In [ ]:
def search_result(model, model_name, X, y, param_grid):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(X ,y)
    print(grid_search.best_params_)

In [ ]:
ds_config = {
    'fake_eps': ['2:1', '4:1'],
    'min_delay': 0.5,
    'max_delay': 5,
    'max_bw': 100,
    'cong_rate': 0.8,
    'stat_size': 100,
    'avg_weight': [0.1, 0.2, 0.3, 0.4],
}

In [ ]:
# basic test
num_switch = 5

data_files = [
    r'../dataset/{}switches/pcap_link_0.5_no_load_no_attack.txt'.format(num_switch),
    r'../dataset/{}switches/pcap_link_0.5_no_load_attack.txt'.format(num_switch),
    r'../dataset/{}switches/pcap_link_0.5_load_attack.txt'.format(num_switch),]

data_size = 14*3000

processors = [
    DataProcess(data_file, data_size, ds_config) for data_file in data_files]

for p in processors:
    p.preprocess()

delays = [p.query_delay_map() for p in processors]

In [ ]:
# log TopoGuard+ result
start_range = range(0, 2000)
test_range = range(2000, 3000)

topoguard_plus = TopoGuardPlus(delays[0], start_range, test_range, ['2:1', '4:1'])
y_pred, y_true = topoguard_plus.fit()
show_model_metrics('TopoGuardPlus', y_true, y_pred)

topoguard_plus = TopoGuardPlus(delays[1], start_range, test_range, ['2:1', '4:1'])
y_pred, y_true = topoguard_plus.fit()
show_model_metrics('TopoGuardPlus', y_true, y_pred)

topoguard_plus = TopoGuardPlus(delays[2], start_range, test_range, ['2:1', '4:1'])
y_pred, y_true = topoguard_plus.fit()
show_model_metrics('TopoGuardPlus', y_true, y_pred)

In [ ]:
start_range = range(0, 2000)
test_range = range(2000, 3000)

trust_topo = TrustTopo(delays[0], start_range, test_range, ['2:1', '4:1'])
y_pred, y_true = trust_topo.fit()
show_model_metrics('TrustTopo', y_true, y_pred)

trust_topo = TrustTopo(delays[1], start_range, test_range, ['2:1', '4:1'])
y_pred, y_true = trust_topo.fit()
show_model_metrics('TrustTopo', y_true, y_pred)


trust_topo = TrustTopo(delays[2], start_range, test_range, ['2:1', '4:1'])
y_pred, y_true = trust_topo.fit()
show_model_metrics('TrustTopo', y_true, y_pred)

In [ ]:
train_range = range(0, 2000)
test_range = range(2000, 3000)

link_guard = LinkGuard(delays[0], train_range, test_range, ['2:1', '4:1'])
y_pred, y_true = link_guard.fit()
show_model_metrics('LinkGuard', y_true, y_pred)

link_guard = LinkGuard(delays[1], train_range, test_range, ['2:1', '4:1'])
y_pred, y_true = link_guard.fit()
show_model_metrics('LinkGuard', y_true, y_pred)


link_guard = LinkGuard(delays[2], train_range, test_range, ['2:1', '4:1'])
y_pred, y_true = link_guard.fit()
show_model_metrics('LinkGuard', y_true, y_pred)

In [ ]:
datasets = [processor.query_dataset() for processor in processors]

lr = LinearRegression()
dt = DecisionTreeClassifier(random_state=0)
rf = RandomForestClassifier(random_state=0)
svc = SVC(random_state=0)
knn = KNeighborsClassifier()
mlp = MLPClassifier(random_state=0)

In [ ]:
# Linear regression

y_pred, y_true = query_model_predict(lr, 'lr', datasets[0])
show_model_metrics('lr', y_true, y_pred)

y_pred, y_true = query_model_predict(lr, 'lr', datasets[1])
show_model_metrics('lr', y_true, y_pred)

y_pred, y_true = query_model_predict(lr, 'lr', datasets[2])
show_model_metrics('lr', y_true, y_pred)

In [ ]:
y_pred, y_true = query_model_predict(dt, 'dt', datasets[0])
show_model_metrics('dt', y_true, y_pred)

y_pred, y_true = query_model_predict(dt, 'dt', datasets[1])
show_model_metrics('dt', y_true, y_pred)

y_pred, y_true = query_model_predict(dt, 'dt', datasets[2])
show_model_metrics('dt', y_true, y_pred)

In [ ]:
# random forest

# {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 16} for dataset 0
# {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 20} for dataset 1
# {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 20} for dataset 2
# dataset = datasets[2]
# param_grid = {
#     'n_estimators': [i for i in range(1, 21)],
#     'max_depth': [i for i in range(1, 11)]
# }

# X, y = dataset[:,:-1], dataset[:,-1]
# search_result(rf, 'rf', X, y, param_grid)


rf = RandomForestClassifier(random_state=0, max_depth=10, n_estimators=16)
y_pred, y_true = query_model_predict(rf, 'rf', datasets[0])
show_model_metrics('rf', y_true, y_pred)

rf = RandomForestClassifier(random_state=0, max_depth=10, n_estimators=20)
y_pred, y_true = query_model_predict(rf, 'rf', datasets[1])
show_model_metrics('rf', y_true, y_pred)

rf = RandomForestClassifier(random_state=0, max_depth=10, n_estimators=20)
y_pred, y_true = query_model_predict(rf, 'rf', datasets[2])
show_model_metrics('rf', y_true, y_pred)


In [ ]:
# SVC
# {'C': 5000} for all dataset
# dataset = datasets[2]
# print(len(dataset))
# param_grid = {
#     'C': [i for i in range(1000, 6000, 1000)],
# }
# X, y = dataset[:,:-1], dataset[:,-1]
# search_result(svc, 'svc', X, y, param_grid)

svc = SVC(random_state=0, C=5000)
y_pred, y_true = query_model_predict(svc, 'svc', datasets[0])
show_model_metrics('svc', y_true, y_pred)

svc = SVC(random_state=0, C=5000)
y_pred, y_true = query_model_predict(svc, 'svc', datasets[1])
show_model_metrics('svc', y_true, y_pred)

svc = SVC(random_state=0, C=5000)
y_pred, y_true = query_model_predict(svc, 'svc', datasets[2])
show_model_metrics('svc', y_true, y_pred)


In [ ]:
# knn
# {'n_neighbors': 20} for all dataset

#dataset = datasets[2]
# param_grid = {
#     'n_neighbors': [i for i in range(1, 21)],
# }

# X, y = dataset[:,:-1], dataset[:,-1]
# search_result(knn, 'knn', X, y, param_grid)

knn1 = KNeighborsClassifier(n_neighbors=20)
y_pred, y_true = query_model_predict(knn1, 'knn', datasets[0])
show_model_metrics('knn', y_true, y_pred)

knn2 = KNeighborsClassifier(n_neighbors=20)
y_pred, y_true = query_model_predict(knn2, 'knn', datasets[1])
show_model_metrics('knn', y_true, y_pred)

knn3 = KNeighborsClassifier(n_neighbors=20)
y_pred, y_true = query_model_predict(knn3, 'knn', datasets[2])
show_model_metrics('knn', y_true, y_pred)

In [ ]:
# dataset = datasets[0]

# # dataset 
# # {'activation': 'relu', 'hidden_layer_data_sizes': 8} for dataset 0
# # {'activation': 'tanh', 'hidden_layer_data_sizes': 10} for dataset 1 
# # {'activation': 'tanh', 'hidden_layer_data_sizes': 10} for dataset 2
# param_grid = {
#     'hidden_layer_sizes': [i for i in range(1, 11)],
#     'activation': ['identity', 'logistic', 'tanh', 'relu']
# }

# X, y = dataset[:,:-1], dataset[:,-1]
# search_result(mlp, 'mlp', X, y, param_grid)

mlp = MLPClassifier(activation='tanh', hidden_layer_sizes=10)
y_pred, y_true = query_model_predict(mlp, 'mlp', datasets[0])
show_model_metrics('mlp', y_true, y_pred)

mlp = MLPClassifier(activation='tanh', hidden_layer_sizes=10)
y_pred, y_true = query_model_predict(mlp, 'mlp', datasets[1])
show_model_metrics('mlp', y_true, y_pred)

mlp = MLPClassifier(activation='tanh', hidden_layer_sizes=10)
y_pred, y_true = query_model_predict(mlp, 'mlp', datasets[2])
show_model_metrics('mlp', y_true, y_pred)


In [ ]:
num_switch = 90
# period_size = 2 * num_switch + 4 
period_size = 2 * 4 # number of fake switches * number of links
data_file = r'../dataset/{}switches/pcap_link_0.5_load_attack.txt'.format(num_switch)
data_size = 10000000
processor = DataProcess(data_file, data_size, ds_config)
processor.preprocess()

delay_map = processor.query_delay_map()
dataset = processor.query_dataset()
start_range = range(0, 2000)
test_range = range(2000, 3000)

In [ ]:
# log topoguard+ result
for s in range(0, 5100, 100):
    start_range = range(s, s+2000)
    test_range = range(s+2000, s+3000)

    topoguard_plus = TopoGuardPlus(delay_map, start_range, test_range, ['2:1', '4:1'])
    y_pred, y_true = topoguard_plus.fit()
    TPR, FPR, Pr, F1, AUC = show_model_metrics('topoguard+', y_true, y_pred)
    with open('topoguard_log/topoguard_log_{}.txt'.format(num_switch), 'a+') as fp:
        fp.write('TPR:{},FPR:{},Pr:{},F1:{},AUC:{}\n'.format(
            TPR, FPR, Pr, F1, AUC
        ))


In [ ]:
# log trustopo result
for s in range(0, 5100, 100):
    start_range = range(s, s+2000)
    test_range = range(s+2000, s+3000)

    trust_topo = TrustTopo(delay_map, start_range, test_range, ['2:1', '4:1'])
    y_pred, y_true = trust_topo.fit()

    TPR, FPR, Pr, F1, AUC = show_model_metrics('trust_topo', y_true, y_pred)
    with open('trusttopo_log/trusttopo_log_{}.txt'.format(num_switch), 'a+') as fp:
        fp.write('TPR:{},FPR:{},Pr:{},F1:{},AUC:{}\n'.format(
            TPR, FPR, Pr, F1, AUC
        ))


In [ ]:
# log linkguard result
for s in range(0, 5100, 100):
    train_range = range(s, s+2000)
    test_range = range(s+2000, s+3000)

    link_guard = LinkGuard(delay_map, train_range, test_range, ['2:1', '4:1'])
    y_pred, y_true = link_guard.fit()
    
    TPR, FPR, Pr, F1, AUC = show_model_metrics('link_guard', y_true, y_pred)
    with open('linkguard_log/linkguard_log_{}.txt'.format(num_switch), 'a+') as fp:
        fp.write('TPR:{},FPR:{},Pr:{},F1:{},AUC:{}\n'.format(
            TPR, FPR, Pr, F1, AUC
        ))


In [ ]:
# log rlv result

for s in range(0, 5100, 100):
    rf = RandomForestClassifier(random_state=0, max_depth=10, n_estimators=20)
    y_pred, y_true = query_model_predict(rf, 'rf', dataset[s*period_size:(s+3000)*period_size])
    TPR, FPR, Pr, F1, AUC = show_model_metrics('rf', y_true, y_pred)
    with open('rf_log/rf_log_{}.txt'.format(num_switch), 'a+') as fp:
        fp.write('TPR:{},FPR:{},Pr:{},F1:{},AUC:{}\n'.format(
            TPR, FPR, Pr, F1, AUC
        ))

In [ ]:
# log rlv result
for s in range(0, 5100, 100):
    mlp = MLPClassifier(hidden_layer_sizes=10, activation='tanh')
    y_pred, y_true = query_model_predict(mlp, 'mlp', dataset[s*period_size:(s+3000)*period_size])
    TPR, FPR, Pr, F1, AUC = show_model_metrics('mlp', y_true, y_pred)
    with open('mlp_log/mlp_log_{}.txt'.format(num_switch), 'a+') as fp:
        fp.write('TPR:{},FPR:{},Pr:{},F1:{},AUC:{}\n'.format(
            TPR, FPR, Pr, F1, AUC
        ))